In [1]:
!pip install datasets
!pip install torch
!pip install spacy
!python -m spacy download en_core_web_trf
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
2024-01-25 12:16:33.157753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 12:16:33.157814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 12:16:33.159233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 12:16:33.167857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized t

In [2]:
from datasets import load_dataset

flore_200 = load_dataset("facebook/flores", "eng_Latn", split='devtest')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for facebook/flores contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/flores
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

In [3]:
len(flore_200)

1012

In [ ]:
# dataset.to_json("flores-200.json")

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

322798

In [ ]:
# data_files = {"testdev": "flore-200.json"}
# flore_200 = load_dataset("json", data_files="flores-200.json", split="train")

In [4]:
flore_200['sentence'][0:5]

['"We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.',
 'Dr. Ehud Ur, professor of medicine at Dalhousie University in Halifax, Nova Scotia and chair of the clinical and scientific division of the Canadian Diabetes Association cautioned that the research is still in its early days.',
 'Like some other experts, he is skeptical about whether diabetes can be cured, noting that these findings have no relevance to people who already have Type 1 diabetes.',
 'On Monday, Sara Danius, permanent secretary of the Nobel Committee for Literature at the Swedish Academy, publicly announced during a radio program on Sveriges Radio in Sweden the committee, unable to reach Bob Dylan directly about winning the 2016 Nobel Prize in Literature, had abandoned its efforts to reach him.',
 'Danius said, "Right now we are doing nothing. I have called and sent emails to his closest collaborator and received very friendly replies. For now, that is certainly enough."']

In [5]:
import spacy
from spacy.matcher import Matcher

def extract_subjects(doc):
    subjects = []
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ in ["nsubj", "nsubjpass", "csubj", "csubjpass"]:
            subjects.append(chunk.text)
    return subjects


# def extract_locations(doc):
#     locations = []
#     for ent in doc.ents:
#         if ent.label_ == "GPE" or ent.label_ == "LOC":
#             preps = [token.text for token in ent.root.head.lefts if token.dep_ == "prep"]
#             location = ' '.join(preps + [ent.text])
#             locations.append(location)
#     return locations


# def extract_prepositional_phrases(doc):
#     prepositional_phrases = []
#     for token in doc:
#         # Check if the token is a preposition
#         if token.pos_ == "ADP":
#             # Initialize phrase with the preposition
#             phrase = [token.text]
#             # Check the right dependencies of the preposition to find the noun or pronoun it's attached to
#             for right in token.rights:
#                 if right.dep_ in ["pobj", "dobj"]:  # object of the preposition
#                     phrase.append(right.text)
#                     # Check for further right dependencies within the noun phrase
#                     phrase.extend([descendant.text for descendant in right.subtree if descendant != right])
#             # Join the components of the phrase and add to the list
#             prepositional_phrases.append(' '.join(phrase))
#     return prepositional_phrases


def extract_transitive_verbs(doc):
    verbs = []
    matcher = Matcher(nlp.vocab)
    pattern = [{"POS": "VERB"}, {"POS": "DET", "OP": "?"}, {"POS": "NOUN"}]
    matcher.add("transitive_verbs", [pattern])
    matches = matcher(doc)
    for _, start, end in matches:
        verb_phrase = doc[start:end]
        verbs.append(verb_phrase.text)
    return verbs


# Remove the duplicates
def add_to_set(case_insensitive_set, items):
    for item in items:
        lower_item = item.lower()
        if lower_item not in case_insensitive_set:
            case_insensitive_set[lower_item] = item

In [8]:
def get_pos_tags(sentence):
    # Process the sentence
    doc = nlp(sentence)

    # Extract words and their POS tags
    word_info = [(token.text, token.pos_, token.dep_) for token in doc]

    return word_info

sent = "a man is eating pizza in the restaurant."

get_pos_tags(sent)

[('a', 'DET', 'det'),
 ('man', 'NOUN', 'nsubj'),
 ('is', 'AUX', 'aux'),
 ('eating', 'VERB', 'ROOT'),
 ('pizza', 'NOUN', 'dobj'),
 ('in', 'ADP', 'prep'),
 ('the', 'DET', 'det'),
 ('restaurant', 'NOUN', 'pobj'),
 ('.', 'PUNCT', 'punct')]

In [6]:
# Load the transformer-based Spacy model
nlp = spacy.load("en_core_web_trf")

In [12]:
sentence_batch = flore_200['sentence']
total_len = len(sentence_batch)
len(sentence_batch)

1012

In [ ]:
# Initialize dictionaries to store unique items
subjects, transitive_verbs = {}, {}

i = 1
for sentence in sentence_batch:
    doc = nlp(sentence)
    # Add items to the respective sets
    add_to_set(subjects, extract_subjects(doc))
    # add_to_set(prep_phrases, extract_prepositional_phrases(doc))
    add_to_set(transitive_verbs, extract_transitive_verbs(doc))
    i += 1

    if (i % 100 == 0):
        print(f"Processed {i} / {total_len}")

In [14]:
# Converting dictionaries back to lists of values
subjects = list(subjects.values())
# locations = list(locations.values())
transitive_verbs = list(transitive_verbs.values())

# Writing to files
with open('subjects.txt', 'w') as file:
    for subject in subjects:
        file.write(subject + '\n')
print("Subjects writing complete.")

# with open('locations.txt', 'w') as file:
#     for location in locations:
#         file.write(location + '\n')
# print("Locations writing complete.")

with open('transitive_verbs.txt', 'w') as file:
    for verb in transitive_verbs:
        file.write(verb + '\n')
print("Transitive Verbs writing complete.")

# print("Subjects:", subjects)
# print("Locations:", locations)
# print("Transitive Verbs:", transitive_verbs)

Subjects writing complete.
Transitive Verbs writing complete.


In [17]:
with open("locs.txt", 'r')  as file1:
    locs = [line.strip() for line in file1.readlines()]

print(locs[:10])

['on shopping television channel QVC', 'television', 'in the field', 'in preventing death', 'In the PALM trial', 'forward', 'in a safe , positive and empowered environment', 'in a statement delivered via the press secretary', 'on top', 'at police headquarters in Gaziantep , Turkey']


In [18]:
with open("subjects.txt", 'r') as file2:
    subs = [line.strip() for line in file2.readlines()]

print(subs[:10])

['We', 'that', 'he', 'Dr. Ehud Ur', 'the research', 'diabetes', 'these findings', 'who', 'Sara Danius', 'the committee']


In [19]:
with open("transitive_verbs.txt", 'r') as file3:
    tvs = [line.strip() for line in file3.readlines()]

print(tvs[:10])

['have no relevance', 'have Type', 'sent emails', 'remarked the company', 'built a WiFi', 'said sales', 'funding the startup', 'settled a lawsuit', 'competing security', 'existing infection']


In [21]:
for sub in subs[:10]:
    for tv in tvs[:10]:
        for loc in locs[:10]:
            # who + doing what + at where
            sent = sub + ' ' + tv + ' ' + loc
            print(sent)

We have no relevance on shopping television channel QVC
We have no relevance television
We have no relevance in the field
We have no relevance in preventing death
We have no relevance In the PALM trial
We have no relevance forward
We have no relevance in a safe , positive and empowered environment
We have no relevance in a statement delivered via the press secretary
We have no relevance on top
We have no relevance at police headquarters in Gaziantep , Turkey
We have Type on shopping television channel QVC
We have Type television
We have Type in the field
We have Type in preventing death
We have Type In the PALM trial
We have Type forward
We have Type in a safe , positive and empowered environment
We have Type in a statement delivered via the press secretary
We have Type on top
We have Type at police headquarters in Gaziantep , Turkey
We sent emails on shopping television channel QVC
We sent emails television
We sent emails in the field
We sent emails in preventing death
We sent emails 